# CEO-Request Challenge

## 1. Summary of Problem Statement

❓ **How could Olist improve its profit** ❓

### P&L Rules

#### Revenues  
**Sales fees:** Olist takes a **10% cut** on the product price (excl. freight) of each order delivered  
**Subscription fees:** Olist charges **80 BRL by month** per seller

#### Costs
**Reputation costs** *estimated* per order with bad reviews (<= 3 stars)  

💡 In the long term, bad customer experience has business implications: low repeat rate, immediate customer support cost, refunds or unfavorable word of mouth communication. We will assume that we have an estimate measure of the monetary cost for each bad review:
```python
# review_score: cost(BRL)
{'1 star': 100
'2 stars': 50
'3 stars': 40
'4 stars': 0
'5 stars': 0}
```

**IT costs:** Olist's **total cumulated IT Costs** scale with the square root of the total number of sellers that has ever join the platform, as well as the square root of the total cumulated number of products that was ever sold.  

$IT\_costs = \alpha * \sqrt{n\_sellers} + \beta * \sqrt{n\_products}$  
Olist's data team gave us the following values for these scaling parameters:
- $\alpha = 3157.27$
- $\beta = 978.23$

💡 Both number of sellers to manage and sales transaction are costly for IT systems.  
💡 Yet square roots reflect scale-effects: IT-system are often more efficient as they grow bigger.  
💡 Alpha > Beta means that "Olist has a lower IT Cost with few sellers selling a lot of products rather than the opposite  
- with **1000 sellers** and a total of **100 products** sold, the total IT cost accumulates to 109,624 BRL
- with **100 sellers** and a total of **1000 products** sold, the total IT cost accumulates to 62,507 BRL

Finally, The IT department also told you that since the birth of the marketplace, cumulated IT costs have amounted to **500,000 BRL**.

### Key Findings, so far

- `wait_time` is the most significant factor behind low review scores  
- `wait_time` is made up of seller's `delay_to_carrier` + `carrier_delivery_time`.  
- The latter being outside of Olist's direct control, improving it is not a quick-win recommendation
- On the contrary, a better selection of `sellers` can positively impact the `delay_to_carrier` and reduce the number of bad `review_scores` on Olist.
- Comments of the bad reviews showed that some were linked to the seller or to the product itself.

💡 We recommend you to start with the the guided seller analysis in part 2 below  
💪 But feel free to investigate into other hypothesis instead with part 3

## 2. Should Olist remove under-performing sellers from its marketplace? 🕵🏻
*(recommended)*

To analyze the impact of removing the worse sellers from Olist's marketplace, we will perform a **what-if analysis**

👉 **What would have happened if Olist had never accepted these sellers in the first place?**  

*(In practice, it's hard to know in advance who is a good seller, but let's start with this approach and iterate later).*

### 2.1 Data Preparation

Compute, for each `seller_id`, and cumulated since the beginning:
- the `revenues` it brings
- the `review_costs` associated with all its bad reviews
- the resulting `profits` (revenues - costs)

👉 Write down a step-by-step strategy to create the DataFrame you need


⚠️ Don't start from scratch, update your existing package 😉

Starting from the `Seller` class of your `olist` package:

Edit the `get_training_data` method so that the DataFrame it returns contains the fields:
- `revenues`: sum of subscription and sales fees revenues
- `cost_of_reviews`: sum of costs associated with bad reviews
- `profits`: `revenues` - `cost_of_reviews`

### 2.2 What-if Analysis

👉 Time to perform the actual analysis, here are our steps:  

1️⃣ Create a method that will help us update the IT Costs after removing sellers along with the products they sold

2️⃣ Sort sellers by increasing profits

3️⃣ Remove sellers one by one, starting from the one with the lowest profit.
- For each number of sellers to remove, compute the financial impact on Olist global profits.

4️⃣ Find an optimal number of sellers to remove that maximizes either Olist's profit margin or profit.

## 3. Investigate other Approaches 🕵️
*(optional)*

- Should Olist remove the worst performing products / categories from its marketplace entirely?
- Should Olist remove only consistently underperforming sellers, after it has a honeymoon period of few months?
- Should Olist restrict seller/customer pairs between certain states to avoid delays?
- Should Olist acquire new sellers, with some cost assumptions to be suggested?
- ...


## Your turn!

🧺 Keep this notebook tidy! 

🗣 📊 You will present your insights to your favorite TA at the end of this `Communicate` unit 💪

# ANSWER

## DATA PREPARATION

In [18]:
import pandas as pd
import numpy as np
from olist.data import Olist
from olist.order import Order


class Seller:
    def __init__(self):
        # Import data only once
        olist = Olist()
        self.data = olist.get_data()
        self.order = Order()

    def get_revenue_cost_profit(self):
        """
        Returns a DataFrame with:
        'seller_id', 'revenues', 'cost_of_reviews', 'profits'
        """
        # Calculate revenues (subscription + sales fees)
        sales = self.get_sales()
        sales['sales_fee'] = sales['sales'] * 0.1  # 10% cut on product price

        # Assume each seller is subscribed for each month they are active
        active_dates = self.get_active_dates()
        active_dates['subscription_fee'] = active_dates['months_on_olist'] * 80  # 80 BRL/month

        revenues = sales.merge(active_dates[['seller_id', 'subscription_fee']], on='seller_id')
        revenues['total_revenue'] = revenues['sales_fee'] + revenues['subscription_fee']

        # Calculate review costs
        review_scores = self.get_review_score()
        review_scores['cost_of_reviews'] = (
            review_scores['share_of_one_stars'] * 100 +  # 1 star cost
            review_scores['share_of_two_stars'] * 50 +   # 2 stars cost
            review_scores['share_of_three_stars'] * 40    # 3 stars cost
        )

        # Merge the revenues and costs
        result = revenues.merge(review_scores[['seller_id', 'cost_of_reviews']], on='seller_id')

        # Calculate profits
        result['profits'] = result['total_revenue'] - result['cost_of_reviews']

        return result[['seller_id', 'total_revenue', 'cost_of_reviews', 'profits']]

    def get_seller_features(self):
        """
        Returns a DataFrame with:
        'seller_id', 'seller_city', 'seller_state'
        """
        sellers = self.data['sellers'].copy(
        )  # Make a copy before using inplace=True so as to avoid modifying self.data
        sellers.drop('seller_zip_code_prefix', axis=1, inplace=True)
        sellers.drop_duplicates(
            inplace=True)  # There can be multiple rows per seller
        return sellers

    def get_seller_delay_wait_time(self):
        """
        Returns a DataFrame with:
        'seller_id', 'delay_to_carrier', 'wait_time'
        """
        # Get data
        order_items = self.data['order_items'].copy()
        orders = self.data['orders'].query("order_status=='delivered'").copy()

        ship = order_items.merge(orders, on='order_id')

        # Handle datetime
        ship.loc[:, 'shipping_limit_date'] = pd.to_datetime(
            ship['shipping_limit_date'])
        ship.loc[:, 'order_delivered_carrier_date'] = pd.to_datetime(
            ship['order_delivered_carrier_date'])
        ship.loc[:, 'order_delivered_customer_date'] = pd.to_datetime(
            ship['order_delivered_customer_date'])
        ship.loc[:, 'order_purchase_timestamp'] = pd.to_datetime(
            ship['order_purchase_timestamp'])

        # Compute delay and wait_time
        def delay_to_logistic_partner(d):
            days = np.mean(
                (d.order_delivered_carrier_date - d.shipping_limit_date) /
                np.timedelta64(24, 'h'))
            if days > 0:
                return days
            else:
                return 0

        def order_wait_time(d):
            days = np.mean(
                (d.order_delivered_customer_date - d.order_purchase_timestamp)
                / np.timedelta64(24, 'h'))
            return days

        delay = ship.groupby('seller_id')\
                    .apply(delay_to_logistic_partner)\
                    .reset_index()
        delay.columns = ['seller_id', 'delay_to_carrier']

        wait = ship.groupby('seller_id')\
                   .apply(order_wait_time)\
                   .reset_index()
        wait.columns = ['seller_id', 'wait_time']

        df = delay.merge(wait, on='seller_id')

        return df

    def get_active_dates(self):
        """
        Returns a DataFrame with:
        'seller_id', 'date_first_sale', 'date_last_sale', 'months_on_olist'
        """
        orders_approved = self.data['orders'][[
            'order_id', 'order_approved_at'
        ]].dropna()
        orders_sellers = orders_approved.merge(self.data['order_items'],
                                               on='order_id')[[
                                                   'order_id', 'seller_id',
                                                   'order_approved_at'
                                               ]].drop_duplicates()
        orders_sellers["order_approved_at"] = pd.to_datetime(
            orders_sellers["order_approved_at"])

        orders_sellers["date_first_sale"] = orders_sellers["order_approved_at"]
        orders_sellers["date_last_sale"] = orders_sellers["order_approved_at"]
        df = orders_sellers.groupby('seller_id').agg({
            "date_first_sale": min,
            "date_last_sale": max
        })
        df['months_on_olist'] = round(
            (df['date_last_sale'] - df['date_first_sale']) /
            np.timedelta64(1, 'M'))
        return df

    def get_quantity(self):
        """
        Returns a DataFrame with:
        'seller_id', 'n_orders', 'quantity', 'quantity_per_order'
        """
        order_items = self.data['order_items']

        n_orders = order_items.groupby('seller_id')['order_id']\
            .nunique()\
            .reset_index()
        n_orders.columns = ['seller_id', 'n_orders']

        quantity = order_items.groupby('seller_id', as_index=False).agg(
            {'order_id': 'count'})
        quantity.columns = ['seller_id', 'quantity']

        result = n_orders.merge(quantity, on='seller_id')
        result['quantity_per_order'] = result['quantity'] / result['n_orders']
        return result

    def get_sales(self):
        """
        Returns a DataFrame with:
        'seller_id', 'sales'
        """
        return self.data['order_items'][['seller_id', 'price']]\
            .groupby('seller_id')\
            .sum()\
            .rename(columns={'price': 'sales'})

    def get_review_score(self):
        """
        Returns a DataFrame with:
        'seller_id', 'share_of_five_stars', 'share_of_one_stars', 'review_score'
        """

        # $CHALLENGIFY_BEGIN
        orders_reviews = self.order.get_review_score()
        orders_sellers = self.data['order_items'][['order_id', 'seller_id'
                                                   ]].drop_duplicates()

        df = orders_sellers.merge(orders_reviews, on='order_id')
        res = df.groupby('seller_id', as_index=False).agg({
            'dim_is_one_star':
            'mean',
            'dim_is_five_star':
            'mean',
            'review_score':
            'mean'
        })

        res.columns = [
            'seller_id', 'share_of_one_stars', 'share_of_five_stars',
            'review_score'
        ]

        return res
        # $CHALLENGIFY_END

    def get_training_data(self):
        """
        Returns a DataFrame with:
        ['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
        'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist', 'share_of_one_stars',
        'share_of_five_stars', 'review_score', 'n_orders', 'quantity',
        'quantity_per_order', 'sales']
        """

        sales_revenues = self.calculate_sales_revenues()
        subscription_revenues = self.calculate_subscription_revenues()
        total_revenues = sales_revenues + subscription_revenues
        review_costs = self.calculate_review_costs()
        profits = total_revenues - review_costs
        training_set = pd.DataFrame({
            'seller_id': self.data['sellers']['seller_id'],
            'revenues': total_revenues,
            'cost_of_reviews': review_costs,
            'profits': profits
        })

        return training_set


## WHAT IF ANALYSIS

In [30]:
import pandas as pd
import numpy as np
from olist.data import Olist
from olist.order import Order


class Seller:
    def __init__(self):
        # Import data only once
        olist = Olist()
        self.data = olist.get_data()
        self.order = Order()

    def get_order_items(self):
        order_items = pd.read_csv('/mnt/data/olist_order_items_dataset.csv')
        return order_items

    def get_revenue_cost_profit(self):
        """
        Returns a DataFrame with:
        'seller_id', 'revenues', 'cost_of_reviews', 'profits'
        """
        sales = self.get_sales()
        sales['sales_fee'] = sales['sales'] * 0.1  # 10% cut on product price
        active_dates = self.get_active_dates()
        active_dates['subscription_fee'] = active_dates['months_on_olist'] * 80  # 80 BRL/month

        revenues = sales.merge(active_dates[['seller_id', 'subscription_fee']], on='seller_id')
        revenues['total_revenue'] = revenues['sales_fee'] + revenues['subscription_fee']
        review_scores = self.get_review_score()
        review_scores['cost_of_reviews'] = (
            review_scores['share_of_one_stars'] * 100 +  # 1 star cost
            review_scores['share_of_two_stars'] * 50 +   # 2 stars cost
            review_scores['share_of_three_stars'] * 40    # 3 stars cost
        )
        result = revenues.merge(review_scores[['seller_id', 'cost_of_reviews']], on='seller_id')
        result['profits'] = result['total_revenue'] - result['cost_of_reviews']
        return result[['seller_id', 'total_revenue', 'cost_of_reviews', 'profits']]

    def get_seller_features(self):
        """
        Returns a DataFrame with:
        'seller_id', 'seller_city', 'seller_state'
        """
        sellers = self.data['sellers'].copy(
        )  # Make a copy before using inplace=True so as to avoid modifying self.data
        sellers.drop('seller_zip_code_prefix', axis=1, inplace=True)
        sellers.drop_duplicates(
            inplace=True)  # There can be multiple rows per seller
        return sellers

    def get_seller_delay_wait_time(self):
        """
        Returns a DataFrame with:
        'seller_id', 'delay_to_carrier', 'wait_time'
        """
        # Get data
        order_items = self.data['order_items'].copy()
        orders = self.data['orders'].query("order_status=='delivered'").copy()

        ship = order_items.merge(orders, on='order_id')
        ship.loc[:, 'shipping_limit_date'] = pd.to_datetime(
            ship['shipping_limit_date'])
        ship.loc[:, 'order_delivered_carrier_date'] = pd.to_datetime(
            ship['order_delivered_carrier_date'])
        ship.loc[:, 'order_delivered_customer_date'] = pd.to_datetime(
            ship['order_delivered_customer_date'])
        ship.loc[:, 'order_purchase_timestamp'] = pd.to_datetime(
            ship['order_purchase_timestamp'])

        # Compute delay and wait_time
        def delay_to_logistic_partner(d):
            days = np.mean(
                (d.order_delivered_carrier_date - d.shipping_limit_date) /
                np.timedelta64(24, 'h'))
            if days > 0:
                return days
            else:
                return 0

        def order_wait_time(d):
            days = np.mean(
                (d.order_delivered_customer_date - d.order_purchase_timestamp)
                / np.timedelta64(24, 'h'))
            return days

        delay = ship.groupby('seller_id')\
                    .apply(delay_to_logistic_partner)\
                    .reset_index()
        delay.columns = ['seller_id', 'delay_to_carrier']

        wait = ship.groupby('seller_id')\
                   .apply(order_wait_time)\
                   .reset_index()
        wait.columns = ['seller_id', 'wait_time']

        df = delay.merge(wait, on='seller_id')

        return df

    def get_active_dates(self):
        """
        Returns a DataFrame with:
        'seller_id', 'date_first_sale', 'date_last_sale', 'months_on_olist'
        """
        orders_approved = self.data['orders'][[
            'order_id', 'order_approved_at'
        ]].dropna()
        orders_sellers = orders_approved.merge(self.data['order_items'],
                                               on='order_id')[[
                                                   'order_id', 'seller_id',
                                                   'order_approved_at'
                                               ]].drop_duplicates()
        orders_sellers["order_approved_at"] = pd.to_datetime(
            orders_sellers["order_approved_at"])
        orders_sellers["date_first_sale"] = orders_sellers["order_approved_at"]
        orders_sellers["date_last_sale"] = orders_sellers["order_approved_at"]
        df = orders_sellers.groupby('seller_id').agg({
            "date_first_sale": min,
            "date_last_sale": max
        })
        df['months_on_olist'] = round(
            (df['date_last_sale'] - df['date_first_sale']) /
            np.timedelta64(1, 'M'))
        return df

    def get_quantity(self):
        """
        Returns a DataFrame with:
        'seller_id', 'n_orders', 'quantity', 'quantity_per_order'
        """
        order_items = self.data['order_items']

        n_orders = order_items.groupby('seller_id')['order_id']\
            .nunique()\
            .reset_index()
        n_orders.columns = ['seller_id', 'n_orders']

        quantity = order_items.groupby('seller_id', as_index=False).agg(
            {'order_id': 'count'})
        quantity.columns = ['seller_id', 'quantity']

        result = n_orders.merge(quantity, on='seller_id')
        result['quantity_per_order'] = result['quantity'] / result['n_orders']
        return result

    def get_sales(self):
        """
        Returns a DataFrame with:
        'seller_id', 'sales'
        """
        return self.data['order_items'][['seller_id', 'price']]\
            .groupby('seller_id')\
            .sum()\
            .rename(columns={'price': 'sales'})

    def get_review_score(self):
        """
        Returns a DataFrame with:
        'seller_id', 'share_of_five_stars', 'share_of_one_stars', 'review_score'
        """

        # $CHALLENGIFY_BEGIN
        orders_reviews = self.order.get_review_score()
        orders_sellers = self.data['order_items'][['order_id', 'seller_id'
                                                   ]].drop_duplicates()

        df = orders_sellers.merge(orders_reviews, on='order_id')
        res = df.groupby('seller_id', as_index=False).agg({
            'dim_is_one_star':
            'mean',
            'dim_is_five_star':
            'mean',
            'review_score':
            'mean'
        })
        # Rename columns
        res.columns = [
            'seller_id', 'share_of_one_stars', 'share_of_five_stars',
            'review_score'
        ]

        return res
        # $CHALLENGIFY_END

    def get_training_data(self):
        """Compiles and returns the training data with calculated profits."""
        orders_df = pd.read_csv('/Users/francoisgirard/code/francoisgirard51/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv/olist_orders_dataset.csv')
        order_items_df = pd.read_csv('/Users/francoisgirard/code/francoisgirard51/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv/olist_order_items_dataset.csv')
        reviews_df = pd.read_csv('/Users/francoisgirard/code/francoisgirard51/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv/olist_order_reviews_dataset.csv')
        sellers_df = pd.read_csv('/Users/francoisgirard/code/francoisgirard51/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv/olist_sellers_dataset.csv')

        order_items_df['sales_fee'] = order_items_df['price'] * 0.1
        sales_fee_per_seller = order_items_df.groupby('seller_id')['sales_fee'].sum().reset_index()
        sellers_df['active_months'] = sellers_df.apply(
            lambda row: max((datetime.now() - row['date_first_sale']).days // 30, 1),
            axis=1
        )
        sellers_df['subscription_fee'] = sellers_df['active_months'] * 80
        subscription_fee_per_seller = sellers_df[['seller_id', 'subscription_fee']]
        reviews_df['review_cost'] = reviews_df['review_score'].apply(self.get_review_cost)
        review_costs_per_order = reviews_df[['order_id', 'review_cost']]
        order_review_costs = order_items_df.merge(review_costs_per_order, on='order_id')
        review_costs_per_seller = order_review_costs.groupby('seller_id')['review_cost'].sum().reset_index()
        training_set = sellers_df[['seller_id']]
        training_set = training_set.merge(sales_fee_per_seller, on='seller_id', how='left')
        training_set = training_set.merge(subscription_fee_per_seller, on='seller_id', how='left')
        training_set = training_set.merge(review_costs_per_seller, on='seller_id', how='left')
        training_set['revenues'] = training_set['sales_fee'] + training_set['subscription_fee']
        training_set['profits'] = training_set['revenues'] - training_set['review_cost']

        return training_set

    def update_it_costs(self, n_sellers, n_products):
        """
        Calculate the IT costs based on the number of sellers and products.
        """
        alpha = 3157.27  # Given constant for sellers
        beta = 978.23    # Given constant for products
        it_costs = alpha * np.sqrt(n_sellers) + beta * np.sqrt(n_products)
        return it_costs

    def what_if_analysis(self):
        """
        Perform the what-if analysis to compute the financial impact of removing
        underperforming sellers on Olist's global profits.
        """
        df = self.get_training_data()
        if 'profits' not in df.columns:
            raise KeyError("DataFrame must contain 'profits' column")


        sorted_sellers = df.sort_values(by='profits', ascending=True)
        original_it_costs = self.update_it_costs(n_sellers=len(sorted_sellers),
                                                 n_products=df['quantity'].sum())
        original_global_profit = df['profits'].sum() - original_it_costs
        what_if_results = []
        cumulative_removed_profits = 0
        cumulative_removed_quantity = 0

        # Step 3: Remove sellers one by one and compute financial impact
        for index, seller in sorted_sellers.iterrows():
            cumulative_removed_profits += seller['profits']
            cumulative_removed_quantity += df.loc[df['seller_id'] == seller['seller_id'], 'quantity'].sum()

            new_it_costs = self.update_it_costs(n_sellers=len(sorted_sellers) - (index + 1),
                                                n_products=df['quantity'].sum() - cumulative_removed_quantity)
            new_global_profit = df['profits'].sum() - cumulative_removed_profits - new_it_costs

            what_if_results.append({
                'n_sellers_removed': index + 1,
                'updated_global_profit': new_global_profit,
                'profit_change': new_global_profit - original_global_profit
            })

        results_df = pd.DataFrame(what_if_results)
        optimal_number_of_sellers = results_df['profit_change'].idxmax()
        optimal_result = results_df.loc[optimal_number_of_sellers]

        return results_df, optimal_result

    def calculate_sales_revenues(self):
        """
        Calculate and return the total sales revenue for each seller.
        """
        order_items = self.data['order_items']
        order_items['sales_fee'] = order_items['price'] * 0.1
        sales_revenues = order_items.groupby('seller_id')['sales_fee'].sum()
        return sales_revenues

    def calculate_subscription_revenues(self):
        """
        Calculate and return the total subscription revenue for each seller.
        """
        active_dates = self.get_active_dates()
        active_dates['subscription_fee'] = active_dates['months_on_olist'] * 80
        subscription_revenues = active_dates['subscription_fee']
        return subscription_revenues

    def get_review_cost(self, review_score):
        """Returns the cost associated with a review score."""
        review_cost_dict = {
            1: 100,  # cost for 1-star reviews
            2: 50,   # cost for 2-star reviews
            3: 40,   # cost for 3-star reviews
            4: 0,    # no cost for 4-star reviews
            5: 0     # no cost for 5-star reviews
        }
        return review_cost_dict.get(review_score, 0)


## OTHER APPROACHES

### Should Olist remove the worst performing products / categories from its marketplace entirely?

### Should Olist remove only consistently underperforming sellers, after it has a honeymoon period of few months?

### Should Olist restrict seller/customer pairs between certain states to avoid delays?

### Should Olist acquire new sellers, with some cost assumptions to be suggested?